In [1]:
import requests
page = requests.get("http://dataquestio.github.io/web-scraping-pages/simple.html")
page

<Response [200]>

In [2]:
page.status_code

200

In [3]:
page.content

b'<!DOCTYPE html>\n<html>\n    <head>\n        <title>A simple example page</title>\n    </head>\n    <body>\n        <p>Here is some simple content for this page.</p>\n    </body>\n</html>'

In [4]:
# We can use BeautifulSoup library to parse this document, and extract the text from the p tag
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   A simple example page
  </title>
 </head>
 <body>
  <p>
   Here is some simple content for this page.
  </p>
 </body>
</html>


In [8]:
# We can first select all the elements at the top level of the page using the children property of soup. Note that children
# return a list generator, so we need to call the list function on it:
list(soup.children)

['html',
 '\n',
 <html>
 <head>
 <title>A simple example page</title>
 </head>
 <body>
 <p>Here is some simple content for this page.</p>
 </body>
 </html>]

In [9]:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.NavigableString, bs4.element.Tag]

In [10]:
# We can now select the html tag and its children by taking the third item in the list
html = list(soup.children)[2]

In [11]:
# Each item in the list returned by the children property is also a BeautifulSoup object, so we can also call the children method on html
list(html.children)

['\n',
 <head>
 <title>A simple example page</title>
 </head>,
 '\n',
 <body>
 <p>Here is some simple content for this page.</p>
 </body>,
 '\n']

In [12]:
body = list(html.children)[3]

In [13]:
# Now, we can get the p tag by finding the children of the body tag:
list(body.children)

['\n', <p>Here is some simple content for this page.</p>, '\n']

In [14]:
# We can now isolate the p tag
p = list(body.children)[1]

In [15]:
# Once we have isolated the tag, we can use the get_text method to extract all of the text inside the tag:
p.get_text()

'Here is some simple content for this page.'

In [16]:
# Finding all instances of a tag at once. If we want to extract a single tag, we can instead use the find_all method, 
# which will find all the instances of a tag on a page
soup = BeautifulSoup(page.content, 'html.parser')
soup.find_all('p')

[<p>Here is some simple content for this page.</p>]

In [17]:
# Note that find_all returns a list, so we will have to loop through, or use list indexing, it to extract text:
soup.find_all('p')[0].get_text()

'Here is some simple content for this page.'

In [19]:
# If you instead only want to find the first instance of a tag, you can use the find method, which will return a single 
# BeautifulSoup object
soup.find('p')

<p>Here is some simple content for this page.</p>

In [20]:
# Searching for tags by class and id
# We wil use a different web page to demonstrate this
page = requests.get("http://dataquestio.github.io/web-scraping-pages/ids_and_classes.html")
soup = BeautifulSoup(page.content, 'html.parser')
soup

<html>
<head>
<title>A simple example page</title>
</head>
<body>
<div>
<p class="inner-text first-item" id="first">
                First paragraph.
            </p>
<p class="inner-text">
                Second paragraph.
            </p>
</div>
<p class="outer-text first-item" id="second">
<b>
                First outer paragraph.
            </b>
</p>
<p class="outer-text">
<b>
                Second outer paragraph.
            </b>
</p>
</body>
</html>

In [21]:
# We can use the find_all method to search for items by class or by id
soup.find_all('p', class_='outer-text')

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>,
 <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [22]:
# To look for any tag that has the class outer-text:
soup.find_all(class_='outer-text')

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>,
 <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [23]:
# We can also search for elements by id:
soup.find_all(id="first")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>]

In [24]:
# BeautifulSoup objects support searching a page via CSS selectors using the select method. We can use CSS selectors to find
# all the p tags in our page that are inside of a div like this:
soup.select("div p")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>,
 <p class="inner-text">
                 Second paragraph.
             </p>]

In [25]:
# Downloading weather data and exploring it using BeautifulSoup
page = requests.get("http://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Tonight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Tonight: Partly cloudy, with a low around 56. West southwest wind 3 to 6 mph. " class="forecast-icon" src="newimages/medium/nsct.png" title="Tonight: Partly cloudy, with a low around 56. West southwest wind 3 to 6 mph. "/>
 </p>
 <p class="short-desc">
  Partly Cloudy
 </p>
 <p class="temp temp-low">
  Low: 56 °F
 </p>
</div>


In [28]:
# There are 4 pieces of information we can extract here
# i) The name of the forecast item - in this case, Tonight
# ii) The description of the conditions - this is stored in the title property of img
# iii) A short description of the conditions - in this case, Partly Cloudy
# iv) The temperature low - in this case, 56 degrees
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()
# Now we can extract the title attribute from the img tag by treating the BeautifulSoup object like a dictionary and pass in the
# attribute we want as a key:
img = tonight.find("img")
desc = img['title']
print(period)
print(short_desc)
print(temp)
print(desc)

Tonight
Partly Cloudy
Low: 56 °F
Tonight: Partly cloudy, with a low around 56. West southwest wind 3 to 6 mph. 


In [29]:
# Extracting all the information from the page
# Here we will select all items with the class period-name inside an item with the class tombstone-container in seven_day
# Use a list comprehension to call the get_text method on each BeautifulSoup object
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Tonight',
 'Tuesday',
 'TuesdayNight',
 'Wednesday',
 'WednesdayNight',
 'Thursday',
 'ThursdayNight',
 'Friday',
 'FridayNight']

In [30]:
# Get the other 3 fields as well
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print(temps)
print(descs)

['Partly Cloudy', 'Sunny', 'Mostly Clear', 'Sunny', 'Mostly Clear', 'Sunny', 'Mostly Clear', 'Sunny', 'Partly Cloudy']
['Low: 56 °F', 'High: 76 °F', 'Low: 57 °F', 'High: 77 °F', 'Low: 56 °F', 'High: 73 °F', 'Low: 54 °F', 'High: 70 °F', 'Low: 55 °F']
['Tonight: Partly cloudy, with a low around 56. West southwest wind 3 to 6 mph. ', 'Tuesday: Sunny, with a high near 76. Calm wind becoming west 5 to 8 mph in the afternoon. ', 'Tuesday Night: Mostly clear, with a low around 57. West southwest wind 5 to 11 mph. ', 'Wednesday: Sunny, with a high near 77. Light west southwest wind increasing to 5 to 10 mph in the afternoon. ', 'Wednesday Night: Mostly clear, with a low around 56. West southwest wind 5 to 8 mph becoming calm  in the evening. ', 'Thursday: Sunny, with a high near 73.', 'Thursday Night: Mostly clear, with a low around 54.', 'Friday: Sunny, with a high near 70.', 'Friday Night: Partly cloudy, with a low around 55.']


In [44]:
# We can now combine our data into a pandas DataFrame
import pandas as pd
weather = pd.DataFrame({
    "period": periods,
    "short_desc": short_descs,
    "temp": temps,
    "desc": descs
})

In [45]:
weather

,period,short_desc,temp,desc
0,Tonight,Partly Cloudy,Low: 56 °F,"Tonight: Partly cloudy, with a low around 56. ..."
1,Tuesday,Sunny,High: 76 °F,"Tuesday: Sunny, with a high near 76. Calm wind..."
2,TuesdayNight,Mostly Clear,Low: 57 °F,"Tuesday Night: Mostly clear, with a low around..."
3,Wednesday,Sunny,High: 77 °F,"Wednesday: Sunny, with a high near 77. Light w..."
4,WednesdayNight,Mostly Clear,Low: 56 °F,"Wednesday Night: Mostly clear, with a low arou..."
5,Thursday,Sunny,High: 73 °F,"Thursday: Sunny, with a high near 73."
6,ThursdayNight,Mostly Clear,Low: 54 °F,"Thursday Night: Mostly clear, with a low aroun..."
7,Friday,Sunny,High: 70 °F,"Friday: Sunny, with a high near 70."
8,FridayNight,Partly Cloudy,Low: 55 °F,"Friday Night: Partly cloudy, with a low around..."


In [51]:
# We can do some analysis here
temp_nums = weather["temp"].str.extract("(\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    56
1    76
2    57
3    77
4    56
5    73
6    54
7    70
8    55
Name: temp, dtype: object

In [52]:
# Mean of all the high and low temperatures
weather["temp_num"].mean()

63.77777777777778

In [53]:
# Select rows that happen at night
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0     True
1    False
2     True
3    False
4     True
5    False
6     True
7    False
8     True
Name: temp, dtype: bool

In [54]:
weather[is_night]

,period,short_desc,temp,desc,temp_num,is_night
0,Tonight,Partly Cloudy,Low: 56 °F,"Tonight: Partly cloudy, with a low around 56. ...",56,True
2,TuesdayNight,Mostly Clear,Low: 57 °F,"Tuesday Night: Mostly clear, with a low around...",57,True
4,WednesdayNight,Mostly Clear,Low: 56 °F,"Wednesday Night: Mostly clear, with a low arou...",56,True
6,ThursdayNight,Mostly Clear,Low: 54 °F,"Thursday Night: Mostly clear, with a low aroun...",54,True
8,FridayNight,Partly Cloudy,Low: 55 °F,"Friday Night: Partly cloudy, with a low around...",55,True
